In [ ]:
# setup
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import (
    ColumnDataSource, Range1d, DataRange1d, DatetimeAxis,
    TickFormatter, DatetimeTickFormatter, FuncTickFormatter,
    Grid, Legend, Plot, BoxAnnotation, Span, CustomJS, Rect, Circle, Line,
    HoverTool, BoxZoomTool, PanTool, WheelZoomTool,
    WMTSTileSource, LabelSet
    )
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Div

import numpy as np
import sys
import os

sys.path.append(os.path.join(os.getcwd(), 'plot_app'))
from pyulog import *
from pyulog.px4 import *
from plotting import *
from config import *
from notebook_helper import *

output_notebook()

In [ ]:
file_name = 'North.ulg' # TODO: fill in file name
ulog = ULog(file_name)
data = ulog.data_list
px4_ulog = PX4ULog(ulog)
px4_ulog.add_roll_pitch_yaw()
use_downsample = False

print("message names: {:}".format(sorted([d.name for d in data])))
print_ulog_info(ulog)

In [ ]:
use_downsample = True # you may want to activate this for large logs (But you will not see all samples when zooming in)

In [ ]:
# load a dataset to inspect the field names and types
sensor_data = [ elem for elem in data if elem.name == 'vehicle_local_position' and elem.multi_id == 0][0]
types_list = [(f.type_str, f.field_name) for f in sensor_data.field_data]
for ftype, fname in types_list: print("{:10s} {:}".format(ftype, fname))

In [ ]:
 # thrust and magnetic field
data_plot = DataPlot(data, plot_config, 'vehicle_magnetometer', y_start=0, title = 'Thrust and Magnetic Field',
                     plot_height='normal')
data_plot.add_graph([lambda data: ('len_mag', 
    np.sqrt(data['magnetometer_ga[0]']**2 + data['magnetometer_ga[1]']**2 + data['magnetometer_ga[2]']**2))],
    colors2[0:1], ['Norm of Magnetic Field'], use_downsample=use_downsample)

#added Theta and Phi 
#theta
data_plot.add_graph([lambda data: ('len_mag', 
    (np.arctan(data['magnetometer_ga[1]']/ + data['magnetometer_ga[0]'])))],
    colors8[6:7], ['Theta of Magnetic Field Radians'], use_downsample=use_downsample)
#phi
data_plot.add_graph([lambda data: ('len_mag', 
    (np.arctan((np.sqrt(data['magnetometer_ga[0]']**2 + data['magnetometer_ga[1]']**2))/data['magnetometer_ga[2]'])))],
    colors8[7:8], ['Phi of Magnetic Field Radians'], use_downsample=use_downsample)
                     

data_plot.change_dataset('actuator_controls_0')
data_plot.add_graph([lambda data: ('thrust', data['control[3]'])], colors2[1:2], ['Thrust'],
                    use_downsample=use_downsample)

show(data_plot.finalize())

In [ ]:
 # Theta and Phi Deg correlation to Thrust 
data_plot = DataPlot(data, plot_config, 'vehicle_magnetometer', y_start=0, title = 'Theta and Phi',
                     plot_height='normal')    

#theta
data_plot.add_graph([lambda data: ('len_mag', 
    np.rad2deg(np.arctan(data['magnetometer_ga[1]']/ + data['magnetometer_ga[0]'])))],
    colors8[6:7], ['Theta of Magnetic Field Degrees'], use_downsample=use_downsample)
#phi
data_plot.add_graph([lambda data: ('len_mag', 
    np.rad2deg(np.arctan((np.sqrt(data['magnetometer_ga[0]']**2 + data['magnetometer_ga[1]']**2))/data['magnetometer_ga[2]'])))],
    colors8[7:8], ['Phi of Magnetic Field Degrees'], use_downsample=use_downsample)
#Thrust
data_plot.change_dataset('actuator_controls_0')
data_plot.add_graph([lambda data: ('thrust',data['control[3]']*100)], colors2[1:2], ['Thrust x 100 (to see correlation)'],
                    use_downsample=use_downsample)  



show(data_plot.finalize())

In [ ]:
 # magnetic field strength
data_plot = DataPlot(data, plot_config, magnetometer_ga_topic,
                     y_axis_label='[gauss]', title='Raw Magnetic Field Strength',
                     plot_height='small', changed_params=changed_params,
                     x_range=x_range)
data_plot.add_graph(['magnetometer_ga[0]', 'magnetometer_ga[1]',
                     'magnetometer_ga[2]'], colors3,
                      ['X', 'Y', 'Z'])
if data_plot.finalize() is not None: plots.append(data_plot)